# Programa de Pós-Graduação em Computação Aplicada (UnB)
## Disciplina: ANÁLISE ESTATÍSTICA DE DADOS E INFORMAÇÕES - Turma 01 - 2023/1
## Professor: João Gabriel de Moraes Souza
## Aluno: Francisco Gonçalves de Araújo Filho
## Matrícula: 231109675
### Prova 1 - Questão 3
A questão 3 irá se basear na aplicação prática de um problema de ANOVA. Esses problemas são originados de dados reais aplicados em problemas de Negócios. A questão irá abordar a aplicação do problema Vendas de Varejo on-line. Esse problema trata-se do objetivo de as **médias de quantidades e preços dos produtos agrupados por países**. Com esses dados pede-se:

### a) Elaborar uma análise descritiva da base de dados, análise gráfica e por tabelas.

In [ ]:
# Importando as bibliotecas
import pandas as pd
import researchpy as rp
import statsmodels.api as sm
from bioinfokit.analys import stat
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon
from scipy.stats import kruskal
import seaborn as sns

In [ ]:
# Carrega dados
cols = ['NumeroFatura', 'CodigoProduto', 'Descricao',
        'Quantidade', 'DataFatura', 'PrecoUnitario',
        'IDCliente', 'Pais']

df = pd.read_csv('ano_2010-2011.csv', encoding='latin1')
df.columns = cols

df.head()

In [ ]:
# Limpa Dados
# Exclui todas faturas canceladas
df = df[~df['NumeroFatura'].str.startswith(('C', 'c'))]

# Exclui linhas que tenham valor e quantidades menores ou igual a zero
df = df[df["Quantidade"] > 0]
df = df[df["PrecoUnitario"] > 0.0]

df.head()

In [ ]:
# Agrupa dados comforme enunciado do exercício
cols_to_group = ['Pais']
df_final = df.groupby(cols_to_group).agg({"Quantidade": "mean", "PrecoUnitario": "mean"})
df_final = df_final.reset_index()
df_final = df_final.dropna()
df_final.head()

In [ ]:
# Gerar um gráfico de dispersão entre Quantidade e PrecoUnitario
import matplotlib.pyplot as plt
plt.scatter(df_final['Quantidade'], df_final['PrecoUnitario'])
plt.xlabel('Média de Quantidade')
plt.ylabel('Média de Preço Unitário')
plt.title('Disperssão Média de Quantidade por Média de Preço Unitário')
plt.show()

#### Análise dos Gráfico de Disperssão
O gráfico de dispersão serve para visualizar a relação entre duas variáveis, neste caso, a media de "Quantidade" vendida e e a média "PrecoUnitario", agrupados por países.
Aparentemente o gráfico não parece ter uma direção de tendência, o que leva a crer não haver relação entre as variáveis.
Identifica-se uma grande concentração de pontos com médias de preço entre 0 e 10 e quantidade média vendida entre 0 e 30.

In [ ]:
# Gráfico de barras para comparar a média das quantidades entre os países
plt.figure(figsize=(12, 6))
sns.barplot(x='Pais', y='Quantidade', data=df_final, errorbar=None)
plt.xticks(rotation=90)
plt.title('Médias das Quantidades por País')
plt.show()

#### Análise do Histograma de Média de Quantidade por país
 Gráfico de barras para comparar a média das quantidades entre os países

 Destaque para média de vendas para Holanda, Suécia, Japão e Austrália

In [ ]:
# Gráfico de barras para comparar a média de Preços Unitários entre os países
plt.figure(figsize=(12, 6))
sns.barplot(x='Pais', y='PrecoUnitario', data=df_final, errorbar=None)
plt.xticks(rotation=90)
plt.title('Médias dos preços unitários de produtos consumidos no País')
plt.ylabel('Preço Unitário Médio')
plt.show()

#### Análise do Histograma de Média de Preços Unitários por país
 Gráfico de barras para comparar a média dos preços unitários de produtos adquiridos pelos países

 Destaque para média de média de preços em Singapura e Hong Kong. O preço médio dos produtos adquidos em Singapura chegam perto de 60 Libras.

In [ ]:
df_lucro = df_final.copy()
df_lucro['Lucro'] = df_lucro['Quantidade'] * df_lucro['PrecoUnitario']

# Criar o gráfico de barras
plt.figure(figsize=(12, 6))
sns.barplot(x='Pais', y='Lucro', data=df_lucro, errorbar=None)
plt.xticks(rotation=90)
plt.title('Estimativa de Lucro Bruto por País')
plt.xlabel('País')
plt.ylabel('Lucro Bruto')
plt.show()

#### Análise do Histograma de Lucro Bruto por país
 Gráfico de barras para comparar a estimativa de lucro por países

 Destaque para Lucro Bruto em Singapura, Hong Kong e Holana. O lucro no período analisado em Singapura chegam perto de 1400 Libras.

### b) Realize a Comparação entre a quantidade e o preço agrupados por países (Use ANOVA)

In [ ]:
# Dropa colunas desnecessárias para o teste
df_ANOVA = df_final.drop(cols_to_group, axis=1)
df_ANOVA.head()

#### Modelo de regressão linear utilizando a função OLS (Ordinary Least Squares) 
 Analisando a relação entre as variáveis

In [ ]:
res = sm.formula.ols('Quantidade ~ PrecoUnitario', data=df_ANOVA).fit()
print(res.summary())

#### Analisando a Regressão
Em resumo, a análise indica que a variável "PrecoUnitario" não tem uma relação significativa com a quantidade vendida. O modelo de regressão não é estatisticamente significativo e a inclusão do preço unitário não melhora a capacidade de explicar a variabilidade da quantidade.

 Neste caso, o valor do F-statistic é 0.03545 e o valor p correspondente é 0.852. Como o valor p é maior que 0.05, não há evidências suficientes para rejeitar a hipótese nula de que o modelo não é significativo.

#### Análise de variância (ANOVA)
Realisa uma análise de variância (ANOVA) usando o modelo de regressão linear ajustado anteriormente.

In [ ]:
anova_table = sm.stats.anova_lm(res, typ=2)
anova_table

#### Analisando resultado ANOVA
O valor p de 0.851714 indica que não há evidências suficientes para rejeitar a hipótese nula de que não existe diferença significativa nas médias da quantidade entre os grupos definidos pelo preço unitário. **Portanto, não há uma relação estatisticamente significativa entre essas variáveis**.

Em resumo, a análise de ANOVA indica que não há diferença significativa nas médias da quantidade entre os grupos definidos pelo preço unitário. O valor p elevado sugere que a variável "PrecoUnitario" não é um fator importante para explicar a variação na quantidade vendida.

### C) Corrigir os problemas dos pressupostos da ANOVA e gerar o modelo mais ajustado para os dados.
Para corrigir os problemas dos pressupostos da ANOVA e obter um modelo mais ajustado para os dados passa-se a verificar a normalidade dos resíduos.



#### QQ-plot (Quantile-Quantile plot) dos resíduos padronizados do modelo ANOVA
Verificar se os resíduos seguem uma distribuição normal

In [ ]:
sm.qqplot(res.resid, line='45')
plt.xlabel("Theoretical Quantiles")
plt.ylabel("Standardized Residuals")
plt.show()

#### Análise do gráfico
O gráfico de QQ-plot mostra a relação entre os quantis teóricos (eixo x) e os resíduos padronizados (eixo y). Se os resíduos seguirem uma distribuição normal, eles devem se alinhar aproximadamente com a linha de referência diagonal (45 graus).

Ao interpretar o gráfico, observe se os pontos dos resíduos estão próximos da linha diagonal. A distribuiçao dos pontos sugere que os resíduos não  seguem uma distribuição normal, isso indica que os resíduos podem não seguir uma distribuição normal, sugerindo uma violação das suposições do modelo.

In [ ]:
plt.hist(res.resid, bins='auto', histtype='bar', ec='k') 
plt.xlabel("Residuals")
plt.ylabel('Frequency')
plt.show()

#### Análise do Histograma:
O histograma mostra a distribuição dos resíduos do modelo ANOVA. O eixo x representa os valores dos resíduos e o eixo y representa a frequência de ocorrência desses valores.

Ao interpretar o histograma, verifica-se que a distribuição não é simétrica. Como a distribuição dos resíduos é assimétrica inica uma violação das suposições do modelo.

#### Teste de Shapiro-Wilk
Realiza o teste de normalidade dos resíduos do modelo estatístico. 

Teste estatístico utilizado para verificar se uma amostra segue uma distribuição normal.

In [ ]:
w, pvalue = stats.shapiro(res.resid)
print('Statistics= %.3f, pvalue=%.3f' % (w, pvalue))
# interpret
alpha = 0.05
if pvalue > alpha:
	print('Resíduo com Distribuição Normal (Não rejeita H0)')
else:
	print('Diferente da Normal (rejeita H0)')

#### Análise do Teste
O Teste de Shapiro-Wilk também sugere uma violação das suposições do modelo, já que o p-valor é menor 0.05 rejeitando a hipotese nula de normalidade da amostra residual do modelo ajustado.

#### Teste Kolmogorov-Smirnov
Teste de Kolmogorov-Smirnov nos resíduos do modelo para verificar se eles seguem uma distribuição normal

In [ ]:
# perform the Kolmogorov-Smirnov test on the residuals
k_statistic, pvalue = stats.kstest(res.resid, 'norm')

# print the results and interpret
alpha = 0.05
if pvalue > alpha:
    print('Resíduo com Distribuição Normal (Não rejeita H0)')
else:
    print('Diferente da Normal (rejeita H0)')

print('K-S Statistic= %.3f, pvalue=%.3f' % (k_statistic, pvalue))

#### Análise do Teste
O Teste de Kolmogorov-Smirnov também sugere uma violação das suposições do modelo, já que o p-valor é menor 0.05 rejeitando a hipotese nula de normalidade da amostra residual do modelo ajustado.

####  Testes Não paramétrico -> Mann-Whitney U Test
teste usado para comparar as distribuições das taxas de retorno entre diferentes grupos. O teste de Mann-Whitney U é um teste não paramétrico utilizado para verificar se duas amostras independentes foram retiradas da mesma população ou se suas distribuições são estatisticamente diferentes.

In [ ]:
stats, p = mannwhitneyu(df_ANOVA["Quantidade"], df_ANOVA["PrecoUnitario"])
print('Statistics= %.3f, p=%.3f' % (stats, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (Não rejeita H0)')
else:
	print('Different distribution (rejeita H0)')

#### Análise da saída
O teste Mann-Whitney U foi aplicado para comparar as distribuições das taxas de retorno entre o retorno da "Quantidade" e o índice "PrecoUnitacio". O resultado apresentado foi:

O valor p é igual a 0.000, que é menor do que alpha (0,05). Isso indica que as evidências disponíveis são suficientes para rejeitar a hipótese nula de que as amostras têm a mesma distribuição. Portanto, podemos afirmar "Different distribution (rejeita H0)", ou seja, as **amostras são provenientes de distribuições diferentes**.

#### Testes Não paramétrico -> Wilcoxon signed-rank

In [ ]:
stats, p = wilcoxon(df_ANOVA["Quantidade"], df_ANOVA["PrecoUnitario"])
print('Statistics= %.3f, p=%.3f' % (stats, p))
# interpret
alpha = 0.05
if p > alpha:
	print('Same distribution (Não rejeita H0)')
else:
	print('Different distribution (rejeita H0)')

#### Análise do Teste
O teste de Wilcoxon, também conhecido como teste de Wilcoxon assinado ou teste de Mann-Whitney assinado. Ele é usado para comparar duas amostras pareadas ou dependentes, quando as observações em cada par não são independentes.

No caso, o valor p é igual a 0.000, que é menor do que alpha (0.05). Isso indica que as evidências disponíveis são suficientes para rejeitar a hipótese nula de que as amostras de 'Quantidade' e 'PrecoUnitario' têm a mesma distribuição. Portanto, podemos afirmar "Different distribution (rejeita H0)", ou seja, as **amostras são provenientes de distribuições diferentes**.

### d) Interpretar os resultados e evidenciar as principais tomadas de decis~oes possívesi com os resultados encontrados.

Todos os testes realizados indicam que **não há uma relação estatisticamente** significativa entre a média de Quantidade de produtos e média do valor os itens vendidos, quando agrupados por país. Portanto as estratégias a serem tomadas não devem levar em consideração essas variáveis.

Entende-se que a empresa responsável pelo comércio a varejo on-line deva investir em propaganda e programa de incentivos para aumentar lucro bruto por país.
Para isso, uma opção é promover produtos mais caras em países que compram mais, como Australia, Japão, Holanda e Suécia, segundo o modelo de comércio aplicando em Singapura, que te o maior preço unitário médio de produtos comercializados naquele país. Já que, apesar de não ser uns dos países mais consumidores do varejo, é Singapura que dá um maior retorno financeiro.